In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
import re
import types
import ast
import sys
sym.init_printing()
sys.path.append(os.path.join(os.path.pardir, 'python'))
from tcg_slb.phasediagram.scipy import ScipyPDReactiveODE
from tcg_slb.phasediagram.base import PDReactiveGrid, PDReactiveGridDiagnostics
from tcg_slb.base import *
import pickle
from pathlib import Path

reference= 'ec_reactions'

In [ ]:
def get_reaction(lib):
    pv = repr(sys.version_info.major)+'.'+repr(sys.version_info.minor)
    sys.path.append(os.path.join(os.path.pardir, 'database', 'install', lib, 'lib', 'python'+pv, 'site-packages/')) # the final slash is necessary!
    tcgdb = __import__('py_'+lib)
    func = getattr(tcgdb,lib) 
    rxn = func() # <-- this should work!
    return rxn

def get_pickle_path(rxnObj): 
    return Path('output', rxnObj.name() + '.pickle')

def load_grid(reactionObject):
    filename = get_pickle_path(reactionObject)
    with open(filename, 'rb') as pfile:
        bdfgrid = pickle.load(pfile)
    return bdfgrid

def save_grid(bdfgrid, reactionObject):
    filename = get_pickle_path(reactionObject)
    filename.parent.mkdir(exist_ok=True, parents=True)
    with open(filename, 'wb+') as pfile:
        pickle.dump(bdfgrid, pfile)

def solve_reaction_grid(reactionObject, mi0=None, Cik0=None, T_range=None, p_range=None, filename=None, reload=False, save=False, odeClass=ScipyPDReactiveODE):
    if reload:
        return load_grid(reactionObject)

    i0 = 2 # doesn't matter as long as you pass cik0
    end = 1e5 # end time

    bdfgrid = PDReactiveGrid()

    bdfgrid.solve(reactionObject, ScipyPDReactiveODE, i0, ['T', 'p'], T_range, p_range, end, Cik0=Cik0, mi0=mi0)

    if save:
        save_grid(bdfgrid, reactionObject)

    return bdfgrid

def plot_reaction_grid(rxn, bdfgrid, t_range, p_range, plot_phases=True,figure_background=None, figure_xlim=[273.15,1273.15], figure_ylim=[0.0,5.0]):
    import matplotlib.pyplot as plt

    def decorate(pdrgd):
        def new_setup_axes(self,axi):
            if(figure_background is not None):
                img = plt.imread(figure_background)
                ip = axi.imshow(img)
                axi.axis('off')
                ax = axi.inset_axes([0.001,0.006,0.995,0.991])
                ax.patch.set_alpha(0.0)
            ax.set_xlim(figure_xlim)
            ax.set_ylim(figure_ylim)
            ax.set_xlabel("Temperature (K)")
            ax.set_ylabel("Pressure (GPa)")
            return ax
    
        #replace the display with newdisplay
        pdrgd.setup_axes = new_setup_axes
    
        #return the modified student 
        return pdrgd

    bdfdiag = decorate(PDReactiveGridDiagnostics)(rxn,bdfgrid) 
    s=bdfdiag.plot_rho()
    s.set_clim([25., 35.])
    s.set_cmap('jet')

    if plot_phases:
        bdfdiag.plot_phases()
    return bdfdiag

def run_reaction(rxnName=None, mi0=None, Cik0=None, t_range=None, p_range=None, reload=False, save=False,t0=900,p0=2.0,figure_background=None,figure_xlim=None,figure_ylim=None):

    rxn = get_reaction(rxnName)

    # initial temperature, pressure and phase volume fraction
    Ti = t0 # kelving
    pi = GPa2Bar(p0) # bars

    try:
        ode = ScipyPDReactiveODE(rxn)
        end_t = 1000.0
        ode.solve(Ti,pi,mi0,Cik0,end_t)
        display(ode.stime)
        display(ode.final_phases(1.e-2))
        ode.plot()
    except:
        pass

    grid = solve_reaction_grid(rxn, mi0=mi0, Cik0=Cik0, T_range=t_range, p_range=p_range, reload=reload, save=save)
    bdfdiag = plot_reaction_grid(rxn, grid, t_range, p_range,figure_background=figure_background, figure_xlim=figure_xlim, figure_ylim=figure_ylim)
    return rxn, grid, bdfdiag

In [ ]:
# Simple eclogite using TCG's Stixrude database

Tmin = 273
Tmax = 1273
nT = 100

Pmin = 0.5
Pmax = 5.0
nP = 100

T = np.linspace(Tmin, Tmax, nT)
P = np.linspace(Pmin, Pmax, nP)

phases = [
    'Clinopyroxene',
    'Orthopyroxene',
    'Quartz',
    'Feldspar', 
    'Garnet', 
    'Kyanite',
]

ems = [
    'Diopside', 'Hedenbergite', 'Clinoenstatite', 'CaTschermaks', 'Jadeite',
    'Enstatite', 'Ferrosilite', 'MgTschermaks', 'OrthoDiopside',
    'Quartz',
    'Anorthite','Albite',
    'Pyrope', 'Almandine', 'Grossular', 'MgMajorite', 'NaMajorite',
    'Kyanite'
]

# mass fractions of the phases

# initialize with eclogitic composition

# 40% cpx, 35% garnet, 15% quartz, 5% kyanite

# Mg# should be around 50

mi0 = np.asarray([
    0.40, # cpx
    0.00, # opx
    0.20, # quartz
    0.00, # feldspar
    0.33, # garnet
    0.07 # kyanite
])

# mass fractions of the end-members
# note: * = thermodynamic endmember, set to zero
Cik0 = np.asarray([
    0.25, 0.25, 0., 0., 0.5, # di, hed, *cEn, *cats, jd
    0.5, 0.5, 0., 0., # en, fs, *mats, *oDi
    1., # quartz
    0.25, 0.75, # an, ab
    0.4, 0.4, 0.2, 0., 0., # py, alm, gr, *mgmaj, *namaj
    1., # ky
])

rxn, grid, bdfdiag = run_reaction(
    "ec_simple_slb_rx",
    mi0=mi0,
    Cik0=Cik0,
    t_range=T,
    p_range=P,
    t0=1200,
    p0=2.5,
    reload=True,
    save=True,
    figure_background="figures/krystopowicz-a4.png",
    figure_xlim = [273.,1273.],
    figure_ylim = [0., 5.,],
)

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
s=bdfdiag.plot_rho_contours()
plt.savefig('figs/density-krystopowicz.pdf')

In [ ]:
rxn, grid, bdfdiag = run_reaction(
    "ec_simple_slb_rx",
    mi0=mi0,
    Cik0=Cik0,
    t_range=T,
    p_range=P,
    t0=1200,
    p0=2.5,
    reload=True,
    save=False,
    figure_background="figures/hp62-cropped.png",
    figure_xlim=[773.0, 1273.0],
    figure_ylim=[0.5, 2.5,],
)

In [ ]:
from matplotlib import pyplot as plt
bdfdiag.plot_phases()
plt.savefig('figs/hp62-pseudosection.pdf')

In [ ]:
rxn, grid, bdfdiag = run_reaction(
    "ec_simple_slb_rx",
    mi0=mi0,
    Cik0=Cik0,
    t_range=T,
    p_range=P,
    t0=1200,
    p0=2.5,
    reload=True,
    save=False,
    figure_background="figures/hp62-density-cropped.png",
    figure_xlim=[773.0, 1273.0],
    figure_ylim=[0.5, 2.5,],
)

s=bdfdiag.plot_rho()
s.set_clim([25., 35.])
s.set_cmap('jet')
plt.savefig('figs/hp62-density.pdf')

In [ ]:
s=bdfdiag.plot_rho_contours()
plt.savefig('figs/hp62-density-contours.pdf')